In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

import sys
sys.path.append('/home/egp/repos/greenland-ird/basis/')

from components.subglacial_drainage_system import SubglacialDrainageSystem
from landlab import RasterModelGrid
from landlab.plot import imshow_grid

In [2]:
def H_of_x(x):
    return 1160 * np.sqrt(1 - x / 10000)

def B_of_x(x):
    return np.where(x < 5000, 100.0, (10000 - x + 1) / 50)

def sliding(grid):
    gamma = 9e-13
    thickness_at_links = grid.map_mean_of_link_nodes_to_link('ice__thickness')
    surface_elevation = grid.at_node['ice__thickness'][:] + grid.at_node['bedrock__elevation'][:]
    driving_stress = 917 * 9.81 * thickness_at_links * grid.calc_grad_at_link(surface_elevation)

    sliding = 20 * np.exp(1 - 1e5 / (driving_stress + 1e3))
    sliding[grid.status_at_link != 0] = 0.0
    return sliding

In [19]:
grid = RasterModelGrid((100, 100), 100.)
grid.status_at_node[grid.boundary_nodes] = grid.BC_NODE_IS_CLOSED

grid.add_field('ice__thickness', H_of_x(grid.node_x), at = 'node')
grid.add_field('bedrock__elevation', B_of_x(grid.node_x), at = 'node')
grid.add_field('ice__sliding_velocity', sliding(grid) / 31556926, at = 'link')
grid.add_field('meltwater__input', np.full(grid.shape, 1.15e-7), at = 'node')

grid.add_field('water__discharge', np.full(grid.number_of_links, 1.15e-7 * np.mean(grid.area_of_patch)), at = 'link')
grid.add_field('effective_pressure', 917 * 9.81 * grid.map_mean_of_link_nodes_to_link('ice__thickness'), at = 'link')
grid.add_field('water__pressure', grid.map_mean_of_links_to_node('effective_pressure') * 0.9, at = 'node')
grid.add_zeros('hydraulic__gradient', at = 'link')
grid.add_zeros('conduit__area', at = 'link')

SDS = SubglacialDrainageSystem(grid)
grid.at_link['hydraulic__gradient'][:] = SDS._calc_base_hydraulic_gradient()

In [20]:
dt = 10
solution = SDS._iter_RK4(SDS._RHS, SDS._build_solution_tensor(to_array = True), dt)
grid.at_link['conduit__area'][:] = np.array(solution[2])

In [22]:
pw = SDS._find_root(SDS._conserve_mass_at_nodes, grid.at_node['water__pressure'][:])

In [33]:
grid.at_node['water__pressure'][:] = pw

grid.at_link['effective_pressure'][:] = SDS._remap(
    SDS._calc_pressure(grid.at_node['water__pressure'][:]), to = 'link'
)

grid.at_link['hydraulic__gradient'][:] = SDS._calc_hydraulic_gradient(
    grid.at_link['effective_pressure'][:]
)

grid.at_link['water__discharge'][:] = SDS._calc_discharge(
    grid.at_link['conduit__area'][:], grid.at_link['hydraulic__gradient'][:]
)

/home/egp/repos/greenland-ird/basis/components/subglacial_drainage_system.py:328: RuntimeWarning: invalid value encountered in power
  np.power(conduit_area, self.params["flow_exp"])
